# Spotify Tracks Test Notebook

In [ ]:
def recommend_system(track_id, data, track_lookup, top_n=5):
    features = data.drop(columns=['track_id']).values
    
    track_idx = np.where(data['track_id'] == track_id)[0][0]
    
    track_features = features[track_idx].reshape(1, -1)

    similarity_scores = cosine_similarity(track_features, features)
    
    similar_indices = np.argsort(similarity_scores[0])[::-1][1:top_n+1]
    
    recommended_tracks = data.iloc[similar_indices]
    
    recommended_tracks_with_names = pd.merge(recommended_tracks, track_lookup, on='track_id', how='left')
    
    return recommended_tracks_with_names

In [ ]:
def get_random_track_id(data):
    return random.choice(data['track_id'].tolist())